In [1]:
import pandas as pd

In [44]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
        df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [43]:
train = read_dataframe('/home/ubuntu/notebooks/data/fhv_tripdata_2021-01.parquet')
test = read_dataframe('/home/ubuntu/notebooks/data/fhv_tripdata_2021-02.parquet')

In [45]:
train[['PUlocationID', 'DOlocationID']] = train[['PUlocationID', 'DOlocationID']].fillna(-1)
test[['PUlocationID', 'DOlocationID']] = test[['PUlocationID', 'DOlocationID']].fillna(-1)

In [35]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [47]:
categorical = ['PUlocationID', 'DOlocationID']
train_dicts = train[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = train['duration']

In [48]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

10.52851910720509

In [49]:
test_dicts = test[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)
y_test = test['duration']

In [50]:
y_pred_test = lr.predict(X_test)

mean_squared_error(y_test, y_pred_test, squared=False)

11.014283137918708